In [1]:
import torch
import torch.nn as nn
from torchvision import datasets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [2]:
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 20
batch_size = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
train_data = datasets.MNIST(root = './data', train = True,
                        transform = transforms.ToTensor(), download = True)

test_data = datasets.MNIST(root = './data', train = False,
                       transform = transforms.ToTensor())

In [4]:
train_gen = torch.utils.data.DataLoader(dataset = train_data,
                                             batch_size = batch_size,
                                             shuffle = True)

test_gen = torch.utils.data.DataLoader(dataset = test_data,
                                      batch_size = batch_size, 
                                      shuffle = False)

In [5]:
class Model(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(Model,self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, num_classes)
  
  def forward(self,x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    return out

In [6]:
model = Model(input_size, hidden_size, num_classes)
model = model.to(device)

Model(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

In [7]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [9]:
for epoch in range(num_epochs):
  for i, (images,labels) in enumerate(train_gen):
    images = Variable(images.view(-1,28*28)).to(device)
    labels = Variable(labels).to(device)
    
    optimizer.zero_grad()
    outputs = model(images)
    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()
    
    if (i+1) % 100 == 0:
      print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, len(train_data)//batch_size, loss.item()))

Epoch [1/20], Step [100/600], Loss: 0.3332
Epoch [1/20], Step [200/600], Loss: 0.2775
Epoch [1/20], Step [300/600], Loss: 0.3627
Epoch [1/20], Step [400/600], Loss: 0.2988
Epoch [1/20], Step [500/600], Loss: 0.1285
Epoch [1/20], Step [600/600], Loss: 0.1771
Epoch [2/20], Step [100/600], Loss: 0.1421
Epoch [2/20], Step [200/600], Loss: 0.1235
Epoch [2/20], Step [300/600], Loss: 0.1509
Epoch [2/20], Step [400/600], Loss: 0.1463
Epoch [2/20], Step [500/600], Loss: 0.1049
Epoch [2/20], Step [600/600], Loss: 0.0894
Epoch [3/20], Step [100/600], Loss: 0.0361
Epoch [3/20], Step [200/600], Loss: 0.0462
Epoch [3/20], Step [300/600], Loss: 0.0234
Epoch [3/20], Step [400/600], Loss: 0.1354
Epoch [3/20], Step [500/600], Loss: 0.0472
Epoch [3/20], Step [600/600], Loss: 0.1115
Epoch [4/20], Step [100/600], Loss: 0.0234
Epoch [4/20], Step [200/600], Loss: 0.0244
Epoch [4/20], Step [300/600], Loss: 0.0321
Epoch [4/20], Step [400/600], Loss: 0.0458
Epoch [4/20], Step [500/600], Loss: 0.0436
Epoch [4/20

In [10]:
correct = 0
total = 0
for images,labels in test_gen:
  images = Variable(images.view(-1,28*28)).to(device)
  labels = labels.to(device)
  
  output = model(images)
  _, predicted = torch.max(output,1)
  correct += (predicted == labels).sum()
  total += labels.size(0)

print('Accuracy of the model: %.3f %%' %((100*correct)/(total+1)))

Accuracy of the model: 98.140 %
